In [22]:
! pip install -Uqq fastbook

In [23]:
import torch
from fastbook import *

In [24]:
path = untar_data(URLs.MNIST_SAMPLE)

In [25]:
Path.BASE_PATH = path

In [26]:
path.ls()

(#3) [Path('train'),Path('labels.csv'),Path('valid')]

In [27]:
(path/'train').ls()

(#2) [Path('train/7'),Path('train/3')]

In [38]:
threes = [tensor(Image.open(o)) for o in (path/'train'/'3').ls().sorted()]
sevens = [tensor(Image.open(o)) for o in (path/'train'/'7').ls().sorted()]

In [51]:
train_x = torch.cat([torch.stack(threes).float()/255, torch.stack(sevens).float()/255]).view(-1, 28*28)
train_x.shape

torch.Size([12396, 784])

In [52]:
train_y = tensor([1,0]*len(threes) + [0, 1]*len(sevens)).view(-1, 2)
train_y.shape

torch.Size([12396, 2])

In [53]:
def binary_cross_entropy(preds, targ):
    preds = preds.sigmoid()
    return -torch.where(targ==1., preds, 1-preds).log().mean()

In [54]:
def init_params(size, std=1.):
    return (torch.randn(size)*std).requires_grad_()

In [101]:
w = init_params((28*28, 2))
b = init_params(2)

In [102]:
w.shape

torch.Size([784, 2])

In [103]:
def linear(xb): return xb@w + b

In [104]:
preds = linear(train_x[:5])
preds

tensor([[ 0.4582, 13.1073],
        [-8.3981,  6.7378],
        [ 4.2423, 17.5388],
        [-1.1856,  6.2586],
        [-7.6100, 13.1619]], grad_fn=<AddBackward0>)

In [105]:
train_y[:5]

tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])

In [106]:
binary_cross_entropy(linear(train_x[:5]), train_y[:5])

tensor(inf, grad_fn=<NegBackward0>)

In [107]:
def step(xb, yb, model):
    preds = model(xb)
    loss = binary_cross_entropy(preds, yb)
    loss.backward()

In [108]:
step(train_x, train_y, linear)

w.data -= w.grad*1.
b.data -= b.grad*1.

w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [109]:
def batch_accuracy(preds, targ):
    preds = preds.sigmoid()
    return ((preds > 0.5) == targ.bool()).float().mean()

In [110]:
batch_accuracy(linear(train_x[:5]), train_y[:5])

tensor(0.5000)

In [111]:
preds = linear(train_x[:5])
preds

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddBackward0>)